In [75]:
import numpy as np
import pandas as pd
import string
import codecs
pd.set_option('display.max_columns', None)

### Problem 1

In [91]:
class hmm(object):
    def __init__(self):
        pass

    def _forward(self, obs):
        """
        Compute the scaled forward probability matrix and scaling factors.
        Parameters
        ----------
        obs : ndarray of shape (T,)
            The observation sequence
        Returns
        -------
        alpha : ndarray of shape (T,N)
            The scaled forward probability matrix
        c : ndarray of shape (T,)
            The scaling factors c = [c_1,c_2,...,c_T]
        """
        M,N = self.B.shape
        T = len(obs)
        c, alpha = np.zeros(T), np.zeros((T,N))
        c[0] = 1/np.dot(self.pi, self.B[obs[0]])
        alpha[0] = c[0]*(self.pi*self.B[obs[0]])
        for t in range(1, T):
            c[t] = 1/np.dot(np.dot(self.A, alpha[t-1]), self.B[obs[t]])
            alpha[t] = c[t]*(np.dot(self.A, alpha[t-1])*self.B[obs[t]])
        return alpha, c
    
    def _backward(self, obs, c):
        """
        Compute the scaled backward probability matrix.
         Parameters
        ----------
        obs : ndarray of shape (T,)
            The observation sequence
        c : ndarray of shape (T,)
            The scaling factors from the forward pass
        Returns
        -------
        beta : ndarray of shape (T,N)
            The scaled backward probability matrix
        """
        M,N = self.B.shape
        T = len(obs)
        beta = np.zeros((T,N))
        beta[T-1] = c[T-1]
        for t in range(T-2,-1,-1):
            beta[t] = np.dot(c[t]*self.A.T,(self.B[obs[t+1]]*beta[t+1]))
        return beta
    def _delta(self, obs, alpha, beta):
        """
        Compute the delta probabilities.
        Parameters
        ----------
        obs : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T,N)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T,N)
            The scaled backward probability matrix from the backward pass
        Returns
        -------
        delta : ndarray of shape (T-1,N,N)
            The delta probability array
        gamma : ndarray of shape (T,N)
            The gamma probability array
        """
        M,N = self.B.shape
        T = len(obs)
        delta = np.zeros((T-1, N, N))
        gamma = np.zeros((T, N))
        
        for t in range(T-1):  
            for i in range(N):
                for j in range(N):
                    delta[t,i,j] = alpha[t,i]*self.A[j,i]*self.B[obs[t+1],j]*beta[t+1,j]
                gamma[t,i] = np.sum(delta[t,i])
        gamma[T-1] = alpha[T-1]*beta[T-1]/np.dot(alpha[T-1], beta[T-1])
        return delta, gamma
    def _estimate(self, obs, delta, gamma):
        """
        Estimate better parameter values.
        Parameters
        ----------
        obs : ndarray of shape (T,)
            The observation sequence
        delta : ndarray of shape (T-1,N,N)
            The delta probability array
        gamma : ndarray of shape (T,N)
            The gamma probability array
        """
        # update self.A, self.B, self.pi in place
        M,N = self.B.shape
        T = len(obs)
        for i in range(N):
            for j in range(N):
                self.A[i,j] = np.sum(delta[:,j,i])/np.sum(gamma[:-1,j])
        for i in range(M):
            for j in range(N):
                self.B[i,j] = np.sum(gamma[:,j][obs==i])/np.sum(gamma[:,j])
        self.pi = gamma[0,:]
        
    def fit(self, obs, N, A=None, B=None, pi=None, max_iter=100, tol=1e-3):
        """
        Fit the model parameters to a given observation sequence.
        Parameters
        ----------
        obs : ndarray of shape (T,)
            Observation sequence on which to train the model.
        A : stochastic ndarray of shape (N,N)
            Initialization of state transition matrix
        B : stochastic ndarray of shape (M,N)
            Initialization of state observation matrix
        pi : stochastic ndarray of shape (N,)
            Initialization of initial state distribution
        max_iter : integer
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        # initialize self.A, self.B, self.pi
        # run the iteration
        M = len(set(obs))
        if A is not None:
            self.A = A/np.sum(A, axis=0) #make sure it is column-stochastic
            N,_ = A.shape
        else:
            self.A = np.random.dirichlet(np.ones(N), size=N).T
        
        if B is not None:
            self.B = B/np.sum(B, axis=0)
        else:
            self.B = np.random.dirichlet(np.ones(M), size=N).T
            
        if pi is not None:
            self.pi = pi/np.sum(pi)
        else:
            self.pi = np.random.dirichlet(np.ones(N))
            
        alpha, c = self._forward(obs)
        log_lkhood0 = - np.sum(np.log(c))
        for i in range(max_iter):
            if i%10==0:
                print("Iter: %d, log_lkhood: %f"%(i, log_lkhood0))
            alpha, c = self._forward(obs)
            beta = self._backward(obs, c)
            delta, gamma = self._delta(obs, alpha, beta)
            
            #update params
            self._estimate(obs, delta, gamma)
            
            alpha, c = self._forward(obs)
            log_lkhood1 = - np.sum(np.log(c))
            if abs(log_lkhood0-log_lkhood1)< tol:
                break
            log_lkhood0=log_lkhood1


In [77]:
#toy data
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
pi = np.array([.6, .4])
obs = np.array([0, 1, 0, 2])

### Problem 2

In [78]:
#test for forward pass
h = hmm()
h.A = A
h.B = B
h.pi = pi
alpha, c = h._forward(obs)
print(-np.log(c).sum()) # the log prob of observation

-4.6429135909


### Problem 3

In [79]:
#test the backward pass
beta = h._backward(obs, c)
beta

array([[ 3.1361635 ,  2.89939354],
       [ 2.86699344,  4.39229044],
       [ 3.898812  ,  2.66760821],
       [ 3.56816483,  3.56816483]])

### Problem 4

In [80]:
delta, gamma = h._delta(obs, alpha, beta)

In [81]:
delta

array([[[ 0.14166321,  0.0465066 ],
        [ 0.37776855,  0.43406164]],

       [[ 0.17015868,  0.34927307],
        [ 0.05871895,  0.4218493 ]],

       [[ 0.21080834,  0.01806929],
        [ 0.59317106,  0.17795132]]])

In [82]:
gamma

array([[ 0.18816981,  0.81183019],
       [ 0.51943175,  0.48056825],
       [ 0.22887763,  0.77112237],
       [ 0.8039794 ,  0.1960206 ]])

### Problem 5

In [83]:
h._estimate(obs, delta, gamma)
print(h.A)
print(h.B)
print(h.pi)


[[ 0.55807991  0.49898142]
 [ 0.44192009  0.50101858]]
[[ 0.23961928  0.70056364]
 [ 0.29844534  0.21268397]
 [ 0.46193538  0.08675238]]
[ 0.18816981  0.81183019]


### Problem 6

In [84]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)
def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  #and convert to all lower case
    #remove punctuation and newlines
    remove_punct_map = {ord(char): None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)

    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    
    #make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    
    #convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

In [85]:
symbols, obs = prep_data('declaration.txt')

### Problem 7

In [93]:
hmm_ = hmm()
hmm_.fit(obs, N=2, max_iter=200)

Iter: 0, log_lkhood: -27984.694651
Iter: 10, log_lkhood: -22096.451772
Iter: 20, log_lkhood: -21855.613115
Iter: 30, log_lkhood: -21652.715537
Iter: 40, log_lkhood: -21579.418872
Iter: 50, log_lkhood: -21552.504780
Iter: 60, log_lkhood: -21536.276039
Iter: 70, log_lkhood: -21522.869274
Iter: 80, log_lkhood: -21513.910457
Iter: 90, log_lkhood: -21510.120438
Iter: 100, log_lkhood: -21508.393164
Iter: 110, log_lkhood: -21507.256295
Iter: 120, log_lkhood: -21506.391586
Iter: 130, log_lkhood: -21505.756749
Iter: 140, log_lkhood: -21505.322756
Iter: 150, log_lkhood: -21505.042934
Iter: 160, log_lkhood: -21504.869024
Iter: 170, log_lkhood: -21504.762593
Iter: 180, log_lkhood: -21504.697269
Iter: 190, log_lkhood: -21504.656458


In [99]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0.301,0.131,,,,0.235,,,0.001,0.123,,,,,,0.14,,,,,,0.057,,,,0.011,
1,0.046,,0.023,0.044,0.06,,0.043,0.031,0.083,,0.004,0.003,0.055,0.035,0.116,,0.033,0.001,0.102,0.115,0.153,,0.018,0.023,0.002,0.009,0.001


### Problem 8

In [101]:
# hmm with 3 states
hmm_3 =  hmm()
hmm_3.fit(obs, N=3, max_iter=200)

Iter: 0, log_lkhood: -25103.772576
Iter: 10, log_lkhood: -21481.699828
Iter: 20, log_lkhood: -20978.434993
Iter: 30, log_lkhood: -20951.726801
Iter: 40, log_lkhood: -20949.497910
Iter: 50, log_lkhood: -20949.070237
Iter: 60, log_lkhood: -20948.771320
Iter: 70, log_lkhood: -20948.301344
Iter: 80, log_lkhood: -20947.924887
Iter: 90, log_lkhood: -20947.816960
Iter: 100, log_lkhood: -20947.796648
Iter: 110, log_lkhood: -20947.791887
Iter: 120, log_lkhood: -20947.790252
Iter: 130, log_lkhood: -20947.789541
Iter: 140, log_lkhood: -20947.789191
Iter: 150, log_lkhood: -20947.789006
Iter: 160, log_lkhood: -20947.788900
Iter: 170, log_lkhood: -20947.788836
Iter: 180, log_lkhood: -20947.788795
Iter: 190, log_lkhood: -20947.788765


In [103]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_3.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0.359,0.068,,,,0.213,,,0.096,0.113,,0.001,0.001,,,0.101,,,0.003,0.001,0.001,0.038,,,,0.006,
1,,,0.008,0.043,0.086,0.043,0.056,0.043,0.012,,0.005,0.004,0.055,0.039,0.049,,0.024,0.002,0.106,0.129,0.218,,0.025,0.028,0.002,0.021,0.001
2,0.072,0.156,0.044,0.035,,0.023,0.01,0.004,,0.049,,,0.038,0.018,0.209,0.111,0.041,,0.064,0.060,,0.052,,0.01,0.003,,


In [102]:
# hmm with 4 states
hmm_4 =  hmm()
hmm_4.fit(obs, N=4, max_iter=200)

Iter: 0, log_lkhood: -26491.366237
Iter: 10, log_lkhood: -21084.722971
Iter: 20, log_lkhood: -20639.194883
Iter: 30, log_lkhood: -20594.095902
Iter: 40, log_lkhood: -20584.392092
Iter: 50, log_lkhood: -20580.165584
Iter: 60, log_lkhood: -20577.197153
Iter: 70, log_lkhood: -20573.903547
Iter: 80, log_lkhood: -20567.329903
Iter: 90, log_lkhood: -20549.905328
Iter: 100, log_lkhood: -20539.691760
Iter: 110, log_lkhood: -20537.063214
Iter: 120, log_lkhood: -20533.772442
Iter: 130, log_lkhood: -20526.231557
Iter: 140, log_lkhood: -20520.965193
Iter: 150, log_lkhood: -20516.225241
Iter: 160, log_lkhood: -20510.880567
Iter: 170, log_lkhood: -20506.710549
Iter: 180, log_lkhood: -20504.449060
Iter: 190, log_lkhood: -20503.471245


In [104]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_4.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,,0.007,0.004,,0.002,0.663,,,,,,0.001,0.002,,,0.192,,,,0.029,0.044,0.02,,,,0.037,
1,,0.124,0.044,0.05,,0.046,0.015,0.017,,0.056,,0.001,0.028,,0.165,0.138,0.037,,0.021,0.053,0.14,0.043,,0.023,,,
2,0.635,0.107,,,,0.001,,,,0.165,,,,0.01,,0.005,0.01,,0.018,,,0.049,,,,,
3,,,,0.031,0.096,,0.058,0.037,0.134,,0.006,0.004,0.064,0.047,0.056,,0.016,0.002,0.133,0.129,0.117,,0.028,0.019,0.003,0.015,0.002


### Problem 9


In [105]:
symbols, obs = prep_data('WarAndPeace.txt')

In [108]:
hmm_2 =  hmm()
hmm_2.fit(obs, N=2, max_iter=200)

Iter: 0, log_lkhood: -207306.469625
Iter: 10, log_lkhood: -167057.782077
Iter: 20, log_lkhood: -159928.818875
Iter: 30, log_lkhood: -158425.865835
Iter: 40, log_lkhood: -158388.675449
Iter: 50, log_lkhood: -158386.868491
Iter: 60, log_lkhood: -158385.870035
Iter: 70, log_lkhood: -158385.682049
Iter: 80, log_lkhood: -158385.635383
Iter: 90, log_lkhood: -158385.618570
Iter: 100, log_lkhood: -158385.612079
Iter: 110, log_lkhood: -158385.609413
Iter: 120, log_lkhood: -158385.608213
Iter: 130, log_lkhood: -158385.607596
Iter: 140, log_lkhood: -158385.607222
Iter: 150, log_lkhood: -158385.606958
Iter: 160, log_lkhood: -158385.606750
Iter: 170, log_lkhood: -158385.606576
Iter: 180, log_lkhood: -158385.606425
Iter: 190, log_lkhood: -158385.606292


In [109]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_2.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,́,а,б,в,г,д,е,ж,з,и,й,к,л,м,н,о,п,р,с,т,у,ф,х,ц,ч,ш,щ,ъ,ы,ь,э,ю,я,ё
0,0.215,,,0.025,0.066,0.03,0.039,0.018,0.014,0.025,0.002,0.015,0.050,0.072,0.038,0.097,,0.035,0.06,0.051,0.078,,0.002,0.011,0.005,0.017,0.011,0.005,,,0.001,,0.008,0.013,
1,0.088,,0.176,,,,,0.143,,,0.131,,0.001,,,,0.241,0.006,,0.028,,0.059,,,,0.004,,,,0.038,0.043,0.007,0.002,0.033,


In [110]:
hmm_3 =  hmm()
hmm_3.fit(obs, N=3, max_iter=200)

Iter: 0, log_lkhood: -206722.641505
Iter: 10, log_lkhood: -167016.909821
Iter: 20, log_lkhood: -166640.153796
Iter: 30, log_lkhood: -166376.748867
Iter: 40, log_lkhood: -166239.097332
Iter: 50, log_lkhood: -166188.786042
Iter: 60, log_lkhood: -166161.622674
Iter: 70, log_lkhood: -166141.460107
Iter: 80, log_lkhood: -166122.366250
Iter: 90, log_lkhood: -166105.709010
Iter: 100, log_lkhood: -166087.292232
Iter: 110, log_lkhood: -166060.262078
Iter: 120, log_lkhood: -166027.852430
Iter: 130, log_lkhood: -165995.554904
Iter: 140, log_lkhood: -165920.602652
Iter: 150, log_lkhood: -165879.112254
Iter: 160, log_lkhood: -165862.662407
Iter: 170, log_lkhood: -165834.125206
Iter: 180, log_lkhood: -165786.659524
Iter: 190, log_lkhood: -165752.703862


In [111]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_3.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,́,а,б,в,г,д,е,ж,з,и,й,к,л,м,н,о,п,р,с,т,у,ф,х,ц,ч,ш,щ,ъ,ы,ь,э,ю,я,ё
0,0.171,,0.006,0.002,0.055,0.04,0.020,0.064,,,0.030,0.008,0.015,0.023,0.017,0.020,0.212,0.053,0.045,0.052,0.092,0.013,,0.012,0.001,0.021,0.001,,,,0.02,0.006,,0.001,
1,0.124,,0.048,0.048,0.033,,0.048,0.142,0.028,0.001,0.069,0.019,,0.046,0.047,0.098,0.036,,0.052,,,0.051,0.002,0.005,0.007,0.001,0.021,0.01,0.001,0.05,,,0.014,,
2,0.195,,0.180,,0.024,0.005,0.003,,,0.05,0.070,,0.081,0.068,0.006,0.072,,0.006,0.008,0.071,0.031,0.010,0.002,,0.002,0.009,,,,,0.033,,0.005,0.068,


In [112]:
hmm_4 =  hmm()
hmm_4.fit(obs, N=4, max_iter=200)

Iter: 0, log_lkhood: -204826.378984
Iter: 10, log_lkhood: -158387.129848
Iter: 20, log_lkhood: -153366.552658
Iter: 30, log_lkhood: -152717.446563
Iter: 40, log_lkhood: -152568.968634
Iter: 50, log_lkhood: -152506.637600
Iter: 60, log_lkhood: -152475.317134
Iter: 70, log_lkhood: -152454.696123
Iter: 80, log_lkhood: -152438.634036
Iter: 90, log_lkhood: -152426.087514
Iter: 100, log_lkhood: -152417.145420
Iter: 110, log_lkhood: -152411.385141
Iter: 120, log_lkhood: -152407.907331
Iter: 130, log_lkhood: -152405.862370
Iter: 140, log_lkhood: -152404.664465
Iter: 150, log_lkhood: -152403.957874
Iter: 160, log_lkhood: -152403.536247
Iter: 170, log_lkhood: -152403.281592
Iter: 180, log_lkhood: -152403.126194
Iter: 190, log_lkhood: -152403.030619


In [113]:
#probability emission matrix for the letters and the space
B = pd.DataFrame(np.around(hmm_4.B, 3), index=symbols).T
B.replace(B[B==0], " ")

,,́,а,б,в,г,д,е,ж,з,и,й,к,л,м,н,о,п,р,с,т,у,ф,х,ц,ч,ш,щ,ъ,ы,ь,э,ю,я,ё
0,0.504,,,0.004,0.052,0.009,0.019,0.056,,0.01,0.008,0.04,0.034,,0.008,,,0.025,0.002,0.112,0.022,0.001,0.002,0.003,,0.016,,,,,,0.012,0.021,0.039,
1,0.009,,0.203,,,0.001,,0.159,,,0.15,,,,,,0.278,,,,,0.067,,,,,,,,0.043,0.052,,0.003,0.034,
2,0.155,,,0.031,0.054,0.046,0.046,,0.024,0.047,,,0.03,0.156,0.07,0.113,,,0.056,0.03,0.068,,0.002,0.019,0.008,0.019,0.02,0.005,,,,,,,
3,0.076,,,0.035,0.073,0.028,0.042,,0.016,0.016,,,0.071,0.056,0.032,0.154,,0.074,0.104,0.047,0.121,,0.002,0.01,0.006,0.018,0.011,0.008,0.001,,,,,,
